In [22]:
import polars as pl
from datetime import date, timedelta
import random
import string

# Function to generate a date range from 2000 to 2024
def generate_date_range(start_year, end_year):
    start_date = date(start_year, 1, 1)
    end_date = date(end_year, 12, 31)
    delta = end_date - start_date
    return [start_date + timedelta(days=i) for i in range(delta.days + 1)]

# Generate a list of dates from 2000 to 2024
dates = generate_date_range(1961, 2023)

# Generate random groups, values, and set ctr to 1 for each entry
groups = [random.choice(string.ascii_uppercase) for _ in dates]
values = [random.randint(1, 5000000) for _ in dates]
ctrs = [1 for _ in dates]

# Create the DataFrame
df = pl.DataFrame({
    "group": groups,
    "values": values,
    "date": dates,
    "ctr": ctrs
})

df.describe()


statistic,group,values,date,ctr
str,str,f64,str,f64
"""count""","""23010""",23010.0,"""23010""",23010.0
"""null_count""","""0""",0.0,"""0""",0.0
"""mean""",null,2.5089e6,"""1992-07-01""",1.0
"""std""",null,1.4424e6,null,0.0
"""min""","""A""",845.0,"""1961-01-01""",1.0
"""25%""",null,1.251312e6,"""1976-10-01""",1.0
"""50%""",null,2.516663e6,"""1992-07-02""",1.0
"""75%""",null,3.756937e6,"""2008-04-01""",1.0
"""max""","""Z""",4.999871e6,"""2023-12-31""",1.0


In [23]:
# Define a custom function for rolling sum operation on a DataFrame grouped by 'group'
# df = pl.read_parquet("little file for testing polars.parquet")
def rolling_sum(group_df):
    # Selects specific columns and applies a rolling sum on the 'ctr' column with a window of 3
    return group_df.select([
        pl.col("group"),                     # Include the 'group' column
        pl.col("date"),                      # Include the 'date' column
        pl.col("ctr").rolling_sum(window_size="30d",by="date", min_periods=1)  # Apply rolling sum on 'ctr'
    ])

# Group the DataFrame by 'group', apply the custom rolling_sum function, and sort the result
result = df.group_by("group").map_groups(rolling_sum).sort(by=["group","date"], descending=False)

# Display the first 6 rows of the resulting DataFrame to showcase the rolling sum operation
result.head(30)

group,date,ctr
str,date,i64
"""A""",1961-01-03,1
"""A""",1961-03-04,1
"""A""",1961-03-21,2
"""A""",1961-03-24,3
"""A""",1961-04-13,3
"""A""",1961-05-11,2
"""A""",1961-05-31,2
"""A""",1961-06-24,2
"""A""",1961-06-27,3


In [24]:
##trying a lazy frame method of doing this

# lazy_df = pl.scan_parquet("huge file for testing polars.parquet")
lazy_df = df.lazy()
#create a query
q = (
    #invoke the lazy df
    lazy_df
    #lets CRUD some columns
    .with_columns(
        #do a 30 day backward looking sum of the counter column by the date column
        pl.col("ctr").rolling_sum(window_size="30d", min_periods=1, by="date")
        #do the above over the group column, so it will calculate each rolling sum by group
        .over("group")
        #use an alias to ensure no confusion
        .alias("group_ctr_rolling")
    )
    #sort so we can interpret the results easily
    .sort(by=["group","date"], descending=False)
    
)
#invoke the query and put the result into lazy_df with streaming on for best performance
lazy_df = q.collect(streaming=True)

#view results
lazy_df.head(30)

group,values,date,ctr,group_ctr_rolling
str,i64,date,i64,i64
"""A""",3142021,1961-01-03,1,1
"""A""",4226875,1961-03-04,1,1
"""A""",4393606,1961-03-21,1,2
"""A""",4930191,1961-03-24,1,3
"""A""",3776338,1961-04-13,1,3
"""A""",882480,1961-05-11,1,2
"""A""",458607,1961-05-31,1,2
"""A""",1234526,1961-06-24,1,2
"""A""",2015775,1961-06-27,1,3


In [25]:
#import interest rates
df_prime = pl.read_csv("canada_prime_Interest.csv")

#change rate column name
df_prime = df_prime.rename({"IRSTPI01CAM156N": "rate"})

df_prime = (
    df_prime
    .with_columns(
        pl.col("DATE").str.slice(0,7).alias("YYYY-MM"),
        pl.col("DATE").str.strptime(pl.Date, "%Y-%m-%d"),
 
        
    )
)



In [26]:
df_prime.head()

DATE,rate,YYYY-MM
date,f64,str
1960-01-01,5.75,"""1960-01"""
1960-02-01,5.75,"""1960-02"""
1960-03-01,5.75,"""1960-03"""
1960-04-01,5.75,"""1960-04"""
1960-05-01,5.75,"""1960-05"""


In [27]:
#convert ddate to string
df = (
    df
    .with_columns(
        pl.col("date").dt.strftime("%Y-%m").alias("YYYY-MM")
    )
    
)

df.head()

group,values,date,ctr,YYYY-MM
str,i64,date,i64,str
"""Y""",1931809,1961-01-01,1,"""1961-01"""
"""K""",1965674,1961-01-02,1,"""1961-01"""
"""A""",3142021,1961-01-03,1,"""1961-01"""
"""E""",4498859,1961-01-04,1,"""1961-01"""
"""O""",2965856,1961-01-05,1,"""1961-01"""


In [29]:
df = df.join(other=df_prime, left_on="YYYY-MM", right_on="YYYY-MM",how="left",suffix="_Prime")



df.sample(30)

group,values,date,ctr,YYYY-MM,DATE,rate,DATE_Prime,rate_Prime
str,i64,date,i64,str,date,f64,date,f64
"""E""",686732,1993-10-25,1,"""1993-10""",1993-10-01,5.75,1993-10-01,5.75
"""Y""",4986424,1990-01-22,1,"""1990-01""",1990-01-01,13.5,1990-01-01,13.5
"""W""",1362123,2009-02-20,1,"""2009-02""",2009-02-01,3.0,2009-02-01,3.0
"""C""",3499444,2019-12-27,1,"""2019-12""",2019-12-01,3.95,2019-12-01,3.95
"""Z""",1101754,1987-08-08,1,"""1987-08""",1987-08-01,10.0,1987-08-01,10.0
"""T""",745793,1964-09-24,1,"""1964-09""",1964-09-01,5.75,1964-09-01,5.75
"""W""",2468376,2004-12-16,1,"""2004-12""",2004-12-01,4.25,2004-12-01,4.25
"""B""",4477711,2023-02-14,1,"""2023-02""",2023-02-01,6.7,2023-02-01,6.7
"""U""",760324,2005-06-30,1,"""2005-06""",2005-06-01,4.25,2005-06-01,4.25
